<a href="https://colab.research.google.com/github/beitezhu/machinelearning/blob/main/beite_case_2_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## read the data
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import statsmodels.formula.api as smf

df = pd.read_csv('https://raw.githubusercontent.com/amandeep0/IS451/main/data/loans.csv')

In [ ]:
##summarize the data structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CreditPolicy    9578 non-null   int64  
 1   Purpose         9578 non-null   object 
 2   IntRate         9578 non-null   float64
 3   Installment     9578 non-null   float64
 4   LogAnnualInc    9578 non-null   float64
 5   Dti             9578 non-null   float64
 6   Fico            9578 non-null   int64  
 7   DaysWithCrLine  9578 non-null   float64
 8   RevolBal        9578 non-null   int64  
 9   RevolUtil       9578 non-null   float64
 10  InqLast6mths    9578 non-null   int64  
 11  Delinq2yrs      9578 non-null   int64  
 12  PubRec          9578 non-null   int64  
 13  NotFullyPaid    9578 non-null   int64  
dtypes: float64(6), int64(7), object(1)
memory usage: 1.0+ MB


In [ ]:
df.head( )

,CreditPolicy,Purpose,IntRate,Installment,LogAnnualInc,Dti,Fico,DaysWithCrLine,RevolBal,RevolUtil,InqLast6mths,Delinq2yrs,PubRec,NotFullyPaid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


In [ ]:
from sklearn.model_selection import train_test_split

fraction_train = 0.7

df_train,df_test = train_test_split(df, test_size = ( 1 - fraction_train), random_state = 88,stratify=df['NotFullyPaid'])

print(df_train.shape[0])
print(df_test.shape[0])

6704
2874


In [ ]:
value_counts = df_test['NotFullyPaid'].value_counts()
count_zero = value_counts[0]
print(count_zero)
print("accurency_baseline=",count_zero/df_test.shape[0])

2414
accurency_baseline= 0.8399443284620738


In [ ]:
df_test['NotFullyPaid'].value_counts()

0    2414
1     460
Name: NotFullyPaid, dtype: int64

a i）  test set baseline accurency = 0.839

In [ ]:
import statsmodels.formula.api as smf

formula = "NotFullyPaid ~ CreditPolicy	+ Purpose +	IntRate +	Installment +	LogAnnualInc +	Dti	+ Fico +	DaysWithCrLine +	RevolBal +	RevolUtil +	InqLast6mths +	Delinq2yrs +	PubRec"

print(dir(smf))
model = smf.logit(formula, data = df_train)
LogModel_res = model.fit()

print(LogModel_res.summary())

['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'conditional_logit', 'conditional_mnlogit', 'conditional_poisson', 'gee', 'glm', 'glmgam', 'gls', 'glsar', 'logit', 'mixedlm', 'mnlogit', 'negativebinomial', 'nominal_gee', 'ols', 'ordinal_gee', 'phreg', 'poisson', 'probit', 'quantreg', 'rlm', 'wls']
Optimization terminated successfully.
         Current function value: 0.410667
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:           NotFullyPaid   No. Observations:                 6704
Model:                          Logit   Df Residuals:                     6685
Method:                           MLE   Df Model:                           18
Date:                Fri, 10 Nov 2023   Pseudo R-squ.:                 0.06615
Time:                        07:54:19   Log-Likelihood:                -2753.1
converged:                       True   LL-Null:       

a ii) Purpose[T.credit_card]，Purpose[T.debt_consolidation],Purpose[T.major_purchase], Purpose[T.small_business],CreditPolicy,Installment, LogAnnualInc, Fico,RevolBal, and InqLast6mths are significent.



In [ ]:
FICO_A = 700
FICO_B = 710
# Define the coefficient associated with the FICO score

fico_coefficient = -0.0086

# Calculate Logit(A) - Logit(B)
logit_difference = fico_coefficient * (FICO_A - FICO_B)
print('a.iii) Logit(A) - Logit(B) = ', logit_difference)

a.iii) Logit(A) - Logit(B) =  0.086


a.iii) Logit(A) - Logit(B) =  0.086

In [ ]:
# predict score of test set adding column of predict score
df_test['PredictedRisk'] = LogModel_res.predict(df_test)
threshold = 0.5

# Classify based on the threshold
df_test['Predictedfullpaied_or_not'] = (df_test['PredictedRisk'] > threshold).astype(int)

# Calculate the accuracy of the logistic regression model on the test set
from sklearn.metrics import accuracy_score

# Caltulate accuracy
accuracy = accuracy_score(df_test['NotFullyPaid'], df_test['Predictedfullpaied_or_not'])
print('accuracy=',accuracy)

accuracy= 0.8416840640222686


a iv) Accuracy of the logistic regression model on the test set with a threshold of 0.5: accuracy= 0.8416840640222686

the accurency_baseline is 0.8399. The threshold of 0.5 model is better than the baseline model.

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate precision
precision = precision_score(df_test['NotFullyPaid'], df_test['Predictedfullpaied_or_not'])

# Calculate recall
recall = recall_score(df_test['NotFullyPaid'], df_test['Predictedfullpaied_or_not'])

# Calculate F1 score
f1 = f1_score(df_test['NotFullyPaid'], df_test['Predictedfullpaied_or_not'])

# Print the results
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)


Precision: 0.6086956521739131
Recall: 0.030434782608695653
F1 Score: 0.057971014492753624


In [ ]:
from sklearn.metrics import roc_auc_score
df_test['PredictedRisk'] = LogModel_res.predict(df_test)
auc = roc_auc_score(df_test['NotFullyPaid'], df_test['PredictedRisk'])
print(auc)


0.6847267749720831


av) AUC of the logistic regression model on the test set: is 0.684
Precision: 0.6086956521739131
Recall: 0.030434782608695653
F1 Score: 0.057971014492753624

In [ ]:
formula = "NotFullyPaid ~ IntRate"

model = smf.logit(formula, data = df_train)
LogModel_IntRate_res = model.fit()

print(LogModel_IntRate_res.summary())

Optimization terminated successfully.
         Current function value: 0.428074
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:           NotFullyPaid   No. Observations:                 6704
Model:                          Logit   Df Residuals:                     6702
Method:                           MLE   Df Model:                            1
Date:                Fri, 10 Nov 2023   Pseudo R-squ.:                 0.02657
Time:                        07:54:54   Log-Likelihood:                -2869.8
converged:                       True   LL-Null:                       -2948.1
Covariance Type:            nonrobust   LLR p-value:                 6.025e-36
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.6429      0.169    -21.554      0.000      -3.974      -3.312
IntRate       15.6922      1.

b i) IntRate is significant in the model. It is not significant in the first model.It is likely because the Interest Rate might have been associated with other variables, rendering the values unreliable. In this case, including only one variable ensures that such correlations do not happen.

In [ ]:
predicted_probabilities_IntRate = LogModel_IntRate_res.predict(df_test['IntRate'])

# Calculate the highest predicted probability of a loan not being paid back in full
highest_predicted_probability = max(predicted_probabilities_IntRate)

# Threshold of 0.5 to classify loans
threshold = 0.5
predicted_numbers = (predicted_probabilities_IntRate > threshold).astype(int)

# Count the number of loans predicted not to be paid back in full
loans_predicted_notfullypaid_num = sum(predicted_numbers)

print('highest_predicted_probability is' ,highest_predicted_probability)
print('loans_predicted_notfullypaid_num is ',loans_predicted_notfullypaid_num )

highest_predicted_probability is 0.43852835776617893
loans_predicted_notfullypaid_num is  0


b ii)
1. The highest predicted probability is approximately 0.439.
2. The number of loans predicted as not fully paid is 0.

In [ ]:

df_test['PredictedRisk_IntRisk'] = LogModel_IntRate_res.predict(df_test)
auc_roc_IntRate = roc_auc_score(df_test['NotFullyPaid'], df_test['PredictedRisk_IntRisk'])
print(auc_roc_IntRate)

0.6258735276106768


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Add a column for predicted class based on the threshold
df_test['Predicted_Class'] = (df_test['PredictedRisk_IntRisk'] > threshold).astype(int)

# Calculate precision
precision = precision_score(df_test['NotFullyPaid'], df_test['Predicted_Class'])

# Calculate recall
recall = recall_score(df_test['NotFullyPaid'], df_test['Predicted_Class'])

# Calculate F1 score
f1 = f1_score(df_test['NotFullyPaid'], df_test['Predicted_Class'])

# Print the results
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)


Precision: 0.0
Recall: 0.0
F1 Score: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


b.iii) Generally, a superior AUC denotes enhanced model prediction capabilities; however, one must weigh this against the consequent loss in interpretability and the heightened danger of overfitting during the process of model selection.

The AUC of the IntRate model on the test set is 0.6258, it is lower than the model using all the independent varibles. The first model has better predictive accuracy.

Precision: 0.0
Recall: 0.0
F1 Score: 0.0





In [ ]:
import math
profit6year = 10 * math.exp(3 * 0.06)
print('profit =', profit6year)

profit = 11.972173631218102


c i) A 10 investment with an annual interest rate of 6% pay back after
3 years, using continuous compounding of interest is $11.97


c ii) If the investment is paid back in full:
Profit = 11.97 - 10 = 1.97

If the investment is not paid back in full: Profit = -10.00


In [ ]:
import numpy as np

df_test['AmountRepaid'] = 1 * np.exp(df['IntRate'] * 3)
df_test['Profit'] = np.where(df_test['NotFullyPaid'] == 0, df_test['AmountRepaid'] - 1, 0)  # 0 profit if not fully paid

# To find the maximum profit
max_profit = df_test['Profit'].max()
print(f'Maximum profit: ${max_profit:.2f}')

Maximum profit: $0.89


   c.iii) Maximum profit of a 1 dollar investment in any loan in the testing set: $0.89

In [ ]:
#select the data with IntRate higher than 0.15 to a new data sheet caeed HighInterest
HighInterest = df_test[df_test['IntRate'] >= 0.15]

# Sort the HighInterest dataset by predicted risk in ascending order
HighInterest = HighInterest.sort_values(by='PredictedRisk', ascending=True)


#calculate the average profit of a $1 investment
average_profit = HighInterest['Profit'].mean()

#caltulate the proportion not paid back in full
not_fully_paid_proportion = HighInterest['NotFullyPaid'].mean()


print('average_profit =', average_profit)
print('not_fully_paid_proportion = ',not_fully_paid_proportion)

average_profit = 0.4638554709463622
not_fully_paid_proportion =  0.2773722627737226


c iv) Average profit of a $1 investment in one of the high-interest loans is 0.51

Proportion of high-interest loans not paid back in full: 17.00%

In [ ]:
# Sort the loans in the HighInterest dataset by the variable PredictedRisk

# Sort the HighInterest dataset by predicted risk in ascending order
HighInterest = HighInterest.sort_values(by='PredictedRisk', ascending=True)

#Create a new dataset called SelectedLoans that consists of the 100 loans with the smallest values of Predictedisk.
SelectedLoans = HighInterest.head(100)

# Compute the profit from investing $1 in each of these 100 loans
total_profit = SelectedLoans['Profit'].sum()
print(f'Total profit: ${total_profit:.2f}')

# Count the number of selected loans not paid back in full
not_paid_back_count = SelectedLoans['NotFullyPaid'].sum()
print(f'Number of loans not paid back in full: {not_paid_back_count}')

# Compare to simple strategy
simple_strategy_profit = 20.94
print(f'Difference in profit: ${total_profit - simple_strategy_profit:.2f}')

Total profit: $51.70
Number of loans not paid back in full: 17
Difference in profit: $30.76


c iv) Total profit: 51.70

Number of loans not paid back in full: 17

Difference in profit: 30.76

d) One common assumption that may not hold in fnancial situations is the assumption of stationarity, which posits that the underlying data generating process does not change over time. Financial data can be highly volatile and infuenced by numerous external factors, leading to non-stationary data distributions.

As an analyst, improving model performance under such circumstances could involve:

1)Feature Engineering: Create features that capture the changing dynamics of financial markets.

2)Model Updating: Regularly update models with recent data to capture the latest trends.

3)Anomaly Detection: Incorporate anomaly detection to identify and adjust for shifts in data distributions.